# Maps

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores7.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 242 ms, sys: 28.5 ms, total: 270 ms
Wall time: 269 ms


In [3]:
autores_df.shape

(76722, 9)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany,2010,Berlin,Madrid,"(52.52000659999999, 13.404954)","(40.4167754, -3.7037902)"
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico,2009,Mexico City,Madrid,"(19.4326077, -99.133208)","(40.4167754, -3.7037902)"
2,Humo,Spain,Antón Fortes Torres,Spain,2008,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
3,Galleta para perros,Spain,Helen Cooper,United Kingdom,2008,London,Madrid,"(51.5073509, -0.1277583)","(40.4167754, -3.7037902)"
4,Autobio,Spain,Cyril Pedrosa,France,2009,Paris,Madrid,"(48.856614, 2.3522219)","(40.4167754, -3.7037902)"


# Rutas

In [5]:
autores_df['PAIS_AUTOR'].nunique()

236

In [6]:
autores_df['PAIS_AUTOR'].value_counts().head()

United States     12174
Spain              9159
France             7477
United Kingdom     6675
Germany            4664
Name: PAIS_AUTOR, dtype: int64

In [7]:
autores_df['PAIS_EJEMPLAR'].nunique()

121

In [8]:
autores_df['PAIS_EJEMPLAR'].value_counts().head()

Spain            37466
Colombia          9891
Argentina         7289
Mexico            6787
United States     3914
Name: PAIS_EJEMPLAR, dtype: int64

In [9]:
%%time

cycles = []
pairs = []
for i, x in autores_df.iterrows():
    pais_autor = x['PAIS_AUTOR']
    pais_ejemplar = x['PAIS_EJEMPLAR']
    if pais_autor == pais_ejemplar:
        cycles.append((pais_autor, pais_ejemplar))
    else:
        pairs.append((pais_autor, pais_ejemplar))

CPU times: user 10 s, sys: 14.5 ms, total: 10 s
Wall time: 10 s


In [10]:
cycles_c = Counter(cycles)
pairs_c = Counter(pairs)

In [11]:
top_cycles = cycles_c.most_common()
top_pairs = pairs_c.most_common()

In [12]:
top_cycles

[(('Spain', 'Spain'), 7333),
 (('Colombia', 'Colombia'), 3889),
 (('Argentina', 'Argentina'), 1703),
 (('United States', 'United States'), 1650),
 (('Mexico', 'Mexico'), 1554),
 (('France', 'France'), 1075),
 (('Venezuela', 'Venezuela'), 462),
 (('Chile', 'Chile'), 322),
 (('Brazil', 'Brazil'), 279),
 (('Peru', 'Peru'), 274),
 (('Cuba', 'Cuba'), 226),
 (('Italy', 'Italy'), 205),
 (('Ecuador', 'Ecuador'), 143),
 (('Germany', 'Germany'), 136),
 (('United Kingdom', 'United Kingdom'), 126),
 (('Uruguay', 'Uruguay'), 81),
 (('Portugal', 'Portugal'), 67),
 (('Russia', 'Russia'), 49),
 (('Japan', 'Japan'), 38),
 (('England', 'England'), 32),
 (('El Salvador', 'El Salvador'), 21),
 (('Guatemala', 'Guatemala'), 20),
 (('Bolivia', 'Bolivia'), 20),
 (('Costa Rica', 'Costa Rica'), 19),
 (('Switzerland', 'Switzerland'), 17),
 (('Dominican Republic', 'Dominican Republic'), 15),
 (('Norway', 'Norway'), 13),
 (('Panama', 'Panama'), 10),
 (('Nicaragua', 'Nicaragua'), 10),
 (('Canada', 'Canada'), 8),
 (

In [13]:
top_pairs

[(('United States', 'Spain'), 6200),
 (('United Kingdom', 'Spain'), 3889),
 (('France', 'Spain'), 3811),
 (('Germany', 'Spain'), 2574),
 (('Italy', 'Spain'), 2246),
 (('France', 'Argentina'), 1006),
 (('United States', 'Colombia'), 924),
 (('United States', 'Argentina'), 857),
 (('United States', 'Mexico'), 844),
 (('Argentina', 'Spain'), 783),
 (('Spain', 'Colombia'), 645),
 (('United Kingdom', 'England'), 631),
 (('France', 'Mexico'), 582),
 (('Belgium', 'Spain'), 534),
 (('Switzerland', 'Spain'), 515),
 (('Ireland', 'Spain'), 514),
 (('United Kingdom', 'United States'), 499),
 (('United States', 'New York (State)'), 494),
 (('Austria', 'Spain'), 475),
 (('Germany', 'Argentina'), 453),
 (('Canada', 'Spain'), 451),
 (('Spain', 'Mexico'), 442),
 (('United Kingdom', 'Mexico'), 422),
 (('United Kingdom', 'Argentina'), 421),
 (('Italy', 'Argentina'), 419),
 (('United States', 'England'), 401),
 (('Poland', 'Spain'), 400),
 (('Mexico', 'Spain'), 398),
 (('Spain', 'Argentina'), 393),
 (('Ru

# Saving results

In [14]:
# autores_df.to_csv('autores6.csv', index=False)